<a href="https://colab.research.google.com/github/danielsoy/ALOCC-CVPR2018/blob/master/177_semantic_segmentation_made_easy_using_segm_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://youtu.be/J_XSd_u_Yew


<br>
@author: Sreenivas Bhattiprolu<br>
Segmentation MOdels library info:<br>
#https://github.com/qubvel/segmentation_models<br>
#Recommended for colab execution<br>
TensorFlow ==2.1.0<br>
keras ==2.3.1<br>
pip install segmentation-models<br>
For this demo it is working on a local workstation...<br>
Python 3.5<br>
TensorFlow ==1.<br>
keras ==2<br>
Data set link: http://brainiac2.mit.edu/isbi_challenge/home<br>


In [ ]:
import tensorflow as tf
import segmentation_models as sm
import glob
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt

ModuleNotFoundError: No module named 'segmentation_models'

In [ ]:
BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)

esizing images is optional, CNNs are ok with large images

In [ ]:
SIZE_X = 256 #Resize images (height  = X, width = Y)
SIZE_Y = 256

apture training image info as a list

In [ ]:
train_images = []

In [ ]:
for directory_path in glob.glob("membrane/augmented_train_256/aug_img"):
    for img_path in glob.glob(os.path.join(directory_path, "*.png")):
        #print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        #img = cv2.resize(img, (SIZE_Y, SIZE_X))
        #img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        #train_labels.append(label)
#Convert list to array for machine learning processing        
train_images = np.array(train_images)

apture mask/label info as a list

In [ ]:
train_masks = [] 
for directory_path in glob.glob("membrane/augmented_train_256/aug_mask"):
    for mask_path in glob.glob(os.path.join(directory_path, "*.png")):
        mask = cv2.imread(mask_path, 0)       
        #mask = cv2.resize(mask, (SIZE_Y, SIZE_X))
        #mask = cv2.cvtColor(mask, cv2.COLOR_RGB2BGR)
        train_masks.append(mask)
        #train_labels.append(label)
#Convert list to array for machine learning processing          
train_masks = np.array(train_masks)

se customary x_train and y_train variables

In [ ]:
X = train_images
Y = train_masks
Y = np.expand_dims(Y, axis=3) #May not be necessary.. leftover from previous code 

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

preprocess input

In [ ]:
x_train = preprocess_input(x_train)
x_val = preprocess_input(x_val)

define model

In [ ]:
model = sm.Unet(BACKBONE, encoder_weights='imagenet')
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['mse'])

In [ ]:
print(model.summary())

In [ ]:
history=model.fit(x_train, 
          y_train,
          batch_size=8, 
          epochs=10,
          verbose=1,
          validation_data=(x_val, y_val))

ccuracy = model.evaluate(x_val, y_val)<br>
lot the training and validation accuracy and loss at each epoch

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

odel.save('membrane.h5')

In [ ]:
from tensorflow import keras
model = keras.models.load_model('membrane.h5', compile=False)
#Test on a different image
#READ EXTERNAL IMAGE...
test_img = cv2.imread('membrane/test/0.png', cv2.IMREAD_COLOR)       
test_img = cv2.resize(test_img, (SIZE_Y, SIZE_X))
test_img = cv2.cvtColor(test_img, cv2.COLOR_RGB2BGR)
test_img = np.expand_dims(test_img, axis=0)

In [ ]:
prediction = model.predict(test_img)

iew and Save segmented image

In [ ]:
prediction_image = prediction.reshape(mask.shape)
plt.imshow(prediction_image, cmap='gray')
plt.imsave('membrane/test0_segmented.jpg', prediction_image, cmap='gray')